In [2]:
import pandas as pd
import numpy as np
import pickle

In [3]:
data = pd.read_csv('pdmssetw.csv')
data

,Unnamed: 0,MeetTijd,HR(bpm),SpO2(%),NIBP(mmHg),IcNaOk
0,0,[''],[''],[''],[''],0.0
1,1,[''],[''],[''],[''],0.0
2,2,[''],[''],[''],[''],0.0
3,3,[''],[''],[''],[''],0.0
4,4,[''],[''],[''],[''],0.0
...,...,...,...,...,...,...
45537,45537,"['19:14:19', '19:14:49', '19:15:19', '19:15:49...","['', '80', '81', '76', '83', '84', '79', '88',...","['100', '100', '100', '100', '100', '100', '10...","['', '123', '', '', '', '', '', '', '', '', ''...",0.0
45538,45538,"['15:17:54', '15:18:24', '15:18:54', '15:19:00...","['67', '', '68', '68', '68', '68', '71', '67',...","['97', '98', '98', '97', '95', '91', '98', '98...","['', '', '', '', '115', '', '', '', '', '113',...",0.0
45539,45539,"['17:39:54', '17:40:24', '17:40:54', '17:41:24...","['100', '103', '97', '101', '100', '106', '107...","['99', '99', '99', '99', '99', '99', '100', '9...","['', '', '', '', '', '', '', '', '', '', '', '...",0.0
45540,45540,"['19:24:26', '19:24:56', '19:25:25', '19:25:56...","['74', '75', '75', '74', '69', '73', '73', '72...","['98', '98', '96', '96', '97', '98', '98', '98...","['178', '', '', '', '', '', '', '185', '', '',...",1.0


In [4]:
data = data.drop(columns=['Unnamed: 0'])

In [5]:
np.nansum(data['IcNaOk'])/len(data)

0.08104606736638707

In [6]:
import ast
data.MeetTijd = data.MeetTijd.apply(ast.literal_eval)
data['HR(bpm)'] = data['HR(bpm)'].apply(ast.literal_eval).apply(lambda x: np.array(x))
data['SpO2(%)'] = data['SpO2(%)'].apply(ast.literal_eval).apply(lambda x: np.array(x))
data['NIBP(mmHg)'] = data['NIBP(mmHg)'].apply(ast.literal_eval).apply(lambda x: np.array(x))

In [7]:
data['Length_MeetTijd'] = data['MeetTijd'].apply(lambda x: len(x))

In [8]:
data = data[data['Length_MeetTijd'] > 3]
data = data.dropna()

In [9]:
data = data.reset_index(drop=True)

In [10]:
data = data[data['Length_MeetTijd'] <2000]

In [11]:
data['HR(bpm)'] = data['HR(bpm)'].apply(lambda x: np.where(x=='', np.nan, x))
data['SpO2(%)'] = data['SpO2(%)'].apply(lambda x: np.where(x=='', np.nan, x))
data['NIBP(mmHg)'] = data['NIBP(mmHg)'].apply(lambda x: np.where(x=='', np.nan, x))

In [12]:
data['Percentage HR nan'] = data['HR(bpm)'].apply(lambda x: np.isnan(x.astype(float)).sum()/len(x))
data['Percentage SpO2 nan'] = data['SpO2(%)'].apply(lambda x: np.isnan(x.astype(float)).sum()/len(x))
data['Percentage NIBP nan'] = data['NIBP(mmHg)'].apply(lambda x: np.isnan(x.astype(float)).sum()/len(x))

In [105]:
data90 = data[(data['Percentage HR nan']<0.10) & (data['Percentage SpO2 nan']<0.10) & (data['Percentage NIBP nan']<0.10)]
data95 = data[(data['Percentage HR nan']<0.05) & (data['Percentage SpO2 nan']<0.05) & (data['Percentage NIBP nan']<0.05)]
data99 = data[(data['Percentage HR nan']<0.01) & (data['Percentage SpO2 nan']<0.01) & (data['Percentage NIBP nan']<0.01)]
data100 = data[(data['Percentage HR nan']==0) & (data['Percentage SpO2 nan']==0) & (data['Percentage NIBP nan']==0)]

In [14]:
pickle.dump(data, open("clean_data.pk", "wb"))
pickle.dump(data100, open("data100.pk", "wb"))

In [103]:
data90 = data90.apply(lambda x: np.array(pd.Series(x.astype(float)).interpolate()))

ValueError: setting an array element with a sequence.

In [104]:
data90

,MeetTijd,HR(bpm),SpO2(%),NIBP(mmHg),IcNaOk,Length_MeetTijd,Percentage HR nan,Percentage SpO2 nan,Percentage NIBP nan
1,"[15:32:32, 15:33:32, 15:34:33, 15:35:33, 15:36...","[51, 49, 57, 49, 55, 50, 56, 49, 50, 49, 54, 4...","[92, 92, 87.5, 93, 93, 93, 92, 95, 93, 92, 93,...","[118, 118, 118, 120, 120, 120, 120, 120, 120, ...",0.0,146,0.000000,0.006849,0.020548
2,"[08:45:22, 08:46:23, 08:47:23, 08:48:24, 08:49...","[79, 75, 71, 88, 88, 84, 66, 66, 76, 78, 79, 8...","[98, 98, 98, nan, nan, 96, 98, 98, 97, 97, 96,...","[129, 129, 129, 129, nan, nan, nan, 145, 145, ...",0.0,200,0.010000,0.020000,0.015000
3,"[08:40:08, 08:41:08, 08:42:09, 08:43:09, 08:44...","[102, 99, 99, 101, 96, 101, 99, 79, 75, 80, 76...","[93, 95.58, 96.16, 96.08, 97.83, 96.66, 97.75,...","[150, 158, 2, 151, 2, 2, 152, 152, 2, 100, 1, ...",0.0,178,0.005618,0.039326,0.005618
4,"[09:09:53, 09:10:53, 09:11:54, 09:12:54, 09:13...","[56, 57, 56, 58, 58, 57, 57, 63, 59, 55, 59, 5...","[97, 97, 98, 97, 98, 98, 97.5, 98, 98, 98, 98,...","[136, 131, 131, 131, 131, 131, 144, 144, 144, ...",0.0,75,0.000000,0.013333,0.066667
5,"[13:45:16, 13:46:17, 13:47:17, 13:48:18, 13:49...","[79, 84, 78, 78, 79, 79, 81, 79, 78, 80, 81, 7...","[97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 9...","[115, 113, 113, 113, 108, 108, 108, 113, 113, ...",0.0,164,0.006098,0.018293,0.018293
...,...,...,...,...,...,...,...,...,...
8327,"[13:56:40, 13:57:10, 13:57:40, 13:58:11, 13:58...","[nan, nan, nan, nan, nan, 71, 78, 75, 78, 73, ...","[100, 100, 99, 100, 100, 100, 100, 100, 100, 1...","[nan, nan, nan, 138, 138, 138, 138, 138, 138, ...",1.0,126,0.039683,0.023810,0.023810
8329,"[17:26:10, 17:26:40, 17:27:11, 17:27:41, 17:28...","[80, 80, 80, 78, 82, 79, 77, 78, 77, 79, 78, 7...","[99, 100, 99, 99, 99, 99, 99, 99, 99, 99, 99, ...","[90, 90, 94, 94, 94, 94, 94, 94, 94, 94, 94, 9...",1.0,190,0.015789,0.015789,0.026316
8331,"[22:36:56, 22:37:27, 22:37:57, 22:38:27, 22:38...","[72, 71, 70, 70, 68, 68, 67, 66, 65, 65, 65, 6...","[100, 100, 100, 100, 100, 100, 100, 100, 100, ...","[160, 124, 124, 124, 124, 124, 124, 98, 98, 98...",1.0,177,0.022599,0.022599,0.000000
8333,"[19:53:44, 19:54:15, 19:54:45, 19:55:16, 19:55...","[125, 131, 128, 125, 126, 123, 128, 130, 126, ...","[97, 97, 98, 98, 98, 97, 97, 98, 98, 98, 99, 9...","[nan, 119, 119, 119, 119, 119, 119, 121, 121, ...",1.0,176,0.005682,0.005682,0.011364
